In [58]:
import pandas as pd
import numpy as np

In [59]:
readers = pd.read_csv("../data/readers.csv")
readers = readers.rename(columns={"id":"user_id", "art_id":"nzz_id"})
readers.head()

,user_id,nzz_id
0,1,ld.154103
1,1,ld.142559
2,1,1.18331199
3,1,ld.144819
4,1,ld.1293110


In [60]:
articles = pd.read_csv("../data/articles_cleaned.csv", encoding="utf-8")
articles.head()

,nzz_id,author,catchline,department,lead_text,pub_date,title,paragraph
0,ld.149648,Claudia Gabriel,Obligationenfonds mit fixer Laufzeit,Finanzen,Die Idee ist gut: Statt einer einzigen Obligat...,2017-03-09 08:01:21.0,Es gibt noch interessante Varianten,Die Idee ist gut: Statt einer einzigen Obligat...
1,1.18145900,Unknown,Fashion Week New York,Panorama,Zum Auftakt der Fashion Week in New York zeige...,2017-04-11 14:00:29.473,Fashion Week New York,NaN
2,ld.138769,Unknown,E-Banking-Ausfall,Finanzen,Seit Sonntag funktioniert das E-Banking der Po...,2017-01-09 13:55:00.0,Postfinance kämpft mit dem System,Seit Sonntag funktioniert das E-Banking der Po...
3,ld.143700,Unknown,Terror in Frankreich,International,Einen Tag nach dem Angriff auf Soldaten beim P...,2017-02-04 12:50:25.0,Louvre nach Macheten-Angriff wieder geöffnet,Einen Tag nach dem Angriff auf Soldaten beim P...
4,ld.149385,Unknown,Unglück in Panama,Panorama,Bei einem Busunglück in Panama sind 17 Persone...,2017-03-06 07:31:21.0,Bus prallt gegen eine Mauer und stürzt in Fluss,Bei einem Busunglück in Panama sind 17 Persone...


In [61]:
read_counts = readers["user_id"].value_counts(sort=True)
read_counts = read_counts.rename_axis("user_id").reset_index(name="read_count")

# Biorę pod uwagę tylko użytkowników, którzy przeczytali minimum 5 artykułów
min_read_count = 3
read_counts = read_counts[read_counts["read_count"] > min_read_count]

readers = readers[readers["user_id"].isin(read_counts["user_id"])]

In [62]:
# Train/Test split
from sklearn.model_selection import train_test_split

random_state = None
readers_train, readers_test = train_test_split(readers,
                                   stratify=readers["user_id"], 
                                   test_size=0.20,
                                   random_state=12)
                                   
print(f"Train set size {len(readers_train)}")
print(f"test set size {len(readers_test)}")
unique_train = readers_train["nzz_id"].unique()
unique_test = readers_test["nzz_id"].unique()
print(f"n users in test {len(unique_test)}")
print(f"n users in train {len(unique_train)}")


Train set size 22284
test set size 5571
n users in test 3995
n users in train 10046


In [63]:
import sys
sys.path.append('../code')
from cf_model import CFModel
from model_evaluator import ModelEvaluator
from random_model import RandomModel
from implicit_model import ImplicitModel
model_evaluator = ModelEvaluator(k_list = [10])

In [64]:
#cf_recommender_model = CFModel(n_latent_factors=200)
cf_recommender_model = ImplicitModel(n_latent_factors=500, regularization=150, alpha=50, iterations=10)
cf_recommender_model.fit(readers_train, articles=articles)
cf_global_metrics, cf_detailed_results_df = model_evaluator.evaluate_model(cf_recommender_model, readers, readers_train, readers_test, interactions=0)
print('\nGlobal metrics:\n%s' % cf_global_metrics)

999 users processed

Global metrics:
{'modelName': 'implicit_model', 'recall@10': 0.4243654761904762, 'precision@10': 0.04243654761904762, 'f1_score@10': 0.07715735930735931, 'ndcg@10': 0.27505183093866165, 'personalization@10': 0.9978148148148148}


In [65]:
read_articles = articles[articles["nzz_id"].isin(readers[readers["user_id"] == 5]["nzz_id"])]["nzz_id"].values.tolist()
cf_recommender_model.recommend(4,topn=100000, articles_to_ignore=read_articles, verbose=True).head(10)

,recommendation_strength,nzz_id,catchline,paragraph,department,lead_text,pub_date
0,0.737799,ld.147049,Die Indigenen Nordskandinaviens,nZwischen Rentierzucht und Klimakonferenz: In ...,Gesellschaft,Zwischenxa0Rentierzucht und Klimakonferenz:xa0...,2017-02-24 04:30:00.0
1,0.687492,ld.141732,«Digitalwährung»xa0Giracoin,Die Schweizer Gira Financial Group wirbt für G...,Digital,Die Schweizer Gira Financial Group wirbt für G...,2017-01-27 09:10:22.0
2,0.681919,ld.1293241,Forschung,Am Anfang war das Bankivahuhn. Sämtliche rund ...,Archiv,Das Huhn: Augenweide und Gaumenfreude.,2017-05-17 04:00:00.0
3,0.681379,ld.152926,Computersicherheit,NaN,Digital,Apple wird erpresst. Die Firma soll 100'000 Do...,2017-03-22 15:47:24.917
4,0.655398,ld.141745,Neue Gegenkultur,Was zum Teufel ist mit der Rockmusik los? Sie ...,NZZaS,Was zum Teufel ist mit der Rockmusik los? Sie ...,2017-01-25 11:43:00.0
5,0.638573,ld.138548,Nationalsymbol,Der inflationäre Gebrauch nationaler Symbole i...,Meinung,Der inflationäre Gebrauch nationaler Symbole i...,2017-01-08 06:00:00.0
6,0.632273,ld.152940,Verbot von Auftritten ausländischer Politiker,NaN,International,In Österreich steht kein Auftritt eines türkis...,2017-03-22 18:26:11.283
7,0.632273,ld.821045,Eigenmittelverordnung,NaN,Wirtschaft,NaN,2017-04-07 15:54:47.663
8,0.629567,ld.1085571,Venezuela in der Krise,NaN,International,Bei Protesten gegen die sozialistische Regieru...,2017-04-11 03:44:42.192
9,0.629469,1.18179552,ATP World Tour Finals,NaN,Sport,Novak Djokovic verteidigt den Titel in London ...,2017-04-11 14:01:02.003


In [66]:
articles[articles["nzz_id"].isin(readers[readers["user_id"] == 5]["nzz_id"])]


,nzz_id,author,catchline,department,lead_text,pub_date,title,paragraph
881,ld.151297,Unknown,Liechtensteinische Landesbank,Finanzen,Die Liechtensteinische Landesbank (LLB) hat im...,2017-03-14 19:42:30.0,Der LLB wachsen Flügel,Die Liechtensteinische Landesbank (LLB) hat im...
1576,ld.154306,Werner Grundlehner,Die Zukunft der Börse,Finanzen,Die traditionellen Börsen kämpfen gegen global...,2017-03-30 04:30:00.0,Der Aktienhandel wird zum Nebenschauplatz,NaN
3096,ld.152567,Unknown,Challenge League,Sport,Neuchâtel Xamax verhindert gegen Le Mont mit z...,2017-03-20 21:11:54.0,Neuenburg bezwingt Le Mont knapp,Neuchâtel Xamax verhindert gegen Le Mont mit z...
4807,ld.139578,Unknown,VW-Abgas-Skandal,Wirtschaft,Ein am Wochenende in Florida wegen mutmasslich...,2017-01-13 04:11:10.0,Kaution für verhafteten Manager abgelehnt,Ein am Wochenende in Florida wegen mutmasslich...
5475,ld.143485,Anja Burri,Internet-Zensur für Online-Kasinos,NZZaS,"Die Schweiz ist auf bestem Weg, eine Internet-...",2017-02-03 09:29:44.0,Das Spiel ist aus,"Die Schweiz ist auf bestem Weg, eine Internet-..."
6075,ld.151690,"Joana Kelen, Simon Wimmer, Michael Radunski",Atommacht Nordkorea,International,Die Raketentests der vergangenen Jahre zeigen:...,2017-04-05 15:44:33.323,Nordkoreas Raketen im Überblick,NaN
6322,ld.147885,Unknown,Super League,Sport,"St. Gallen gegen Lugano ist, wenn die Heimmann...",2017-02-26 17:14:00.0,Lugano siegt dank Sadiku,"St. Gallen gegen Lugano ist, wenn die Heimmann..."
6500,ld.1294366,Joseph Croitoru,Hamas und Muslimbruderschaft,Meinung,Die palästinensische Hamas hat jüngst ein neue...,2017-05-18 03:30:00.0,Die Nabelschnur zu Ägypten ist gekappt,Die Muslimbruderschaft wurde 1928 mit dem Ziel...
6530,ld.141814,Jochen Siegle,TV-Technik,Digital,Es war absehbar: Nach Samsung verabschieden si...,2017-01-25 15:18:57.0,3D macht sich dünn,Es war absehbar: Nach Samsung verabschieden si...
6822,ld.155220,Christof Leisinger,DAX knapp unter Rekordniveau,Finanzen,Gewagte Strategien zahlen sich an den Finanzmä...,2017-04-04 05:00:00.0,Lateinamerikas und Europas Märkte «ziehen»,NaN
